In [1]:
!nvidia-smi

Wed Feb 12 14:48:32 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 430.26       Driver Version: 430.26       CUDA Version: 10.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  GeForce RTX 208...  Off  | 00000000:03:00.0 Off |                  N/A |
| 27%   36C    P2    45W / 250W |   4683MiB / 11019MiB |      3%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [2]:
%%capture
import tensorflow as tf
from tensorflow.keras.models import model_from_json
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import backend as K
import numpy as np
import h5py

In [3]:
from tensorflow.keras.models import load_model
from utils import read_h5_dataset
from BilinearInterpolate3D import BilinearInterpolate3D
from OrthogonalConv import NetXCycle

W0212 14:50:48.722883 139690189117248 deprecation_wrapper.py:119] From /workspace/icml_submission/KJHYgfOhas/ICML-Submission/OrthogonalConv.py:4: The name tf.keras.initializers.RandomUniform is deprecated. Please use tf.compat.v1.keras.initializers.RandomUniform instead.



In [4]:
!pip3 freeze

absl-py==0.7.1
asn1crypto==0.24.0
astor==0.8.0
attrs==19.3.0
backcall==0.1.0
bleach==3.1.0
cryptography==2.1.4
cycler==0.10.0
decorator==4.4.1
defusedxml==0.6.0
entrypoints==0.3
gast==0.2.2
google-pasta==0.1.7
graphviz==0.13.2
grpcio==1.21.1
h5py==2.9.0
idna==2.6
imageio==2.6.1
imgaug==0.3.0
importlib-metadata==1.5.0
ipykernel==5.1.4
ipython==7.11.1
ipython-genutils==0.2.0
ipywidgets==7.5.1
jedi==0.16.0
Jinja2==2.11.0
joblib==0.14.0
jsonschema==3.2.0
jupyter==1.0.0
jupyter-client==5.3.4
jupyter-console==6.1.0
jupyter-core==4.6.1
Keras-Applications==1.0.8
Keras-Preprocessing==1.1.0
keyring==10.6.0
keyrings.alt==3.0
kiwisolver==1.1.0
Markdown==3.1.1
MarkupSafe==1.1.1
matplotlib==3.1.2
mistune==0.8.4
nbconvert==5.6.1
nbformat==5.0.4
networkx==2.4
notebook==6.0.3
numpy==1.16.4
opencv-python-headless==4.1.2.30
pandas==0.25.3
pandocfilters==1.4.2
parso==0.6.0
pexpect==4.8.0
pickleshare==0.7.5
Pillow==6.2.1
prometheus-client==0.7.1
prompt-toolkit==3.0.3
protobuf==3.8.0
ptyprocess==0.6.0
pycry

In [5]:
BATCH_SIZE = 64

## Load CIFAR10 model

In [6]:
%%capture
cifar_model_path = 'model_val_accuracy.h5'
cifar_dataset_path = 'test_00001.h5'
cifar_model = load_model(cifar_model_path, custom_objects={'BilinearInterpolate3D':BilinearInterpolate3D})

In [7]:
# Compiling the model
opt_rms = RMSprop(lr=0.001,decay=1e-6)
cifar_model.compile(loss='categorical_crossentropy', optimizer=opt_rms, metrics=['accuracy'])

cifar_model.summary()

Model: "encoder"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
encoder_input (InputLayer)   [(None, 32, 32, 3)]       0         
_________________________________________________________________
conv2d (Conv2D)              (None, 32, 32, 80)        2240      
_________________________________________________________________
batch_normalization_v1 (Batc (None, 32, 32, 80)        320       
_________________________________________________________________
activation (Activation)      (None, 32, 32, 80)        0         
_________________________________________________________________
bilinear_interpolate3d (Bili (None, 75, 75, 75)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 75, 75, 75)        50700     
_________________________________________________________________
batch_normalization_v1_1 (Ba (None, 75, 75, 75)        300 

In [8]:
# Load dataset
x_test, y_test = read_h5_dataset(cifar_dataset_path)

In [9]:
# Evaluating 
scores = cifar_model.evaluate(x_test, y_test, batch_size=BATCH_SIZE)

10000/10000 [==============================] - 15s 1ms/sample - loss: 0.4911 - acc: 0.8815


## Load PATHFINDER model

In [7]:
# Specify the model directory path - no trailing `/`
pathfinder_model_dir = 'model_kernel20_path9'

In [8]:
!sh untar.sh $pathfinder_model_dir

!ls model_kernel20_path9/

model_kernel20_path9.h5
model.tar.gz.partaa  model.tar.gz.partab  model_kernel20_path9.h5


In [9]:
%%capture
pathfinder_model_path = pathfinder_model_dir + '/' + pathfinder_model_dir + '.h5'

pathfinder_model = load_model(pathfinder_model_path, custom_objects={'BilinearInterpolate3D':BilinearInterpolate3D,
                                                                        'NetXCycle': NetXCycle})

!rm $pathfinder_model_path

In [10]:
pathfinder_model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 128, 128, 1)]     0         
_________________________________________________________________
conv2d (Conv2D)              (None, 128, 128, 128)     51328     
_________________________________________________________________
batch_normalization_v1 (Batc (None, 128, 128, 128)     512       
_________________________________________________________________
activation (Activation)      (None, 128, 128, 128)     0         
_________________________________________________________________
net_x_cycle (NetXCycle)      (None, 128, 128, 128)     19662720  
_________________________________________________________________
global_max_pooling2d (Global (None, 128)               0         
_________________________________________________________________
flatten (Flatten)            (None, 128)               0     